## Objective Function:

Maximizing the revenue, given as the difference between the reward (profit) and the cost per click times the clicks ( costs ). 

It can be expressed as :
### $ max_{price,bid}( \sum \limits_{C_i} (price \times soldItems(bid, price)) - (clicks(bid) \times costPerClick(bid))) $





(price, bid) for each customer class ; 

#in theory we know, but actually we don't 


In [1]:
from customerManager import CustomerManager
path = "./config/customer_classes.json"
manager = CustomerManager(file_path = path, noise_variances = [100, 100 ,100 ,100])

manager.clicks(class_id = 0, bid= 1)

8.805691102608005